<a href="https://colab.research.google.com/github/telmavcosta/data_processing/blob/main/spark_streaming/challenges/rep_final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00


# Producer

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.option('checkpointLocation', 'content/lake/bronze/messages/checkpoint')
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [4]:
query.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipython-input-3-2779494543.py", line 26, in insert_messages
    enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return

In [5]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/data")
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2025-07-06 08:54:...|    0|   CREATED|2ff7ec01-6ea5-427...|  OTHER|      2012|   1017|
|2025-07-06 08:54:...|    2|   CREATED|2ff7ec01-6ea5-427...|  OTHER|      2012|   1017|
|2025-07-06 08:54:...|    1|   CREATED|2ff7ec01-6ea5-427...|  OTHER|      2012|   1017|
|2025-07-06 08:54:...|    3|   CREATED|2ff7ec01-6ea5-427...|  OTHER|      2012|   1017|
|2025-07-06 08:54:...|   44|  RECEIVED|e53712e3-1e26-4e3...|  EMAIL|      2008|   1001|
|2025-07-06 08:54:...|   21|  RECEIVED|59c1cf52-c13b-4b6...|  EMAIL|      2009|   1037|
|2025-07-06 08:55:...|   80|  RECEIVED|48b329a0-9010-4a1...|  OTHER|      2014|   1019|
|2025-07-06 08:54:...|   35|  RECEIVED|712d7aec-2225-4b9...|  OTHER|      2005|   1007|
|2025-07-06 08:54:...|   46|  RE

86

In [6]:
query.isActive

False

# Additional datasets

In [7]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [ ]:
!rm -rf lake/silver/*
!rm -rf "{'"content/lake/*

In [8]:
# TODO

from pyspark.sql.types import *
#from pyspark.sql.functions import col

bronze_path="content/lake/bronze/messages/data"
silver_non_corrupted_path="content/lake/silver/messages/data"
silver_corrupted_path="content/lake/silver/messages_corrupted/data"

message_schema = StructType([StructField('timestamp', TimestampType(), True),
                            StructField('value', LongType(), True),
                            StructField('event_type', StringType(), True),
                            StructField('message_id', StringType(), True),
                            StructField('channel', StringType(), True),
                            StructField('country_id', IntegerType(), True),
                            StructField('user_id', IntegerType(), True)])

df_bronze = spark.readStream.format("parquet").schema(message_schema).load("content/lake/bronze/messages/data")


enriched_df = df_bronze.join(countries, on="country_id", how="left").withColumn("country", F.col("country")).withColumn("date", F.to_date(F.col("timestamp")))

# Split into corrupted and valid
is_corrupted = (F.col("event_type").isNull() | (F.trim(F.col("event_type")) == "") | (F.trim(F.col("event_type")) == "NONE"))

corrupted_messages_df = enriched_df.filter(is_corrupted)
valid_messages_df = enriched_df.filter(~is_corrupted)


corrupted_query = corrupted_messages_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation", "content/lake/silver/messages_corrupted/checkpoint") \
    .option("path","content/lake/silver/messages_corrupted/data") \
    .partitionBy("date") \
    .trigger(processingTime="5 seconds") \
    .start()

valid_query = valid_messages_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation", "content/lake/silver/messages/checkpoint") \
    .option("path","content/lake/silver/messages/data") \
    .partitionBy("date") \
    .trigger(processingTime="5 seconds") \
    .start()

corrupted_query.awaitTermination(20)
valid_query.awaitTermination(20)


False

In [14]:
df_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data")
df_corrupted.show()
df_corrupted.count()

+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|       country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+----------+
|      2015|2025-07-06 08:55:...|   72|      NONE|573ccde8-e795-420...|  EMAIL|   1011|     Argentina|2025-07-06|
|      2015|2025-07-06 08:54:...|   43|      NONE|8db13d9d-8aaf-467...|   PUSH|   1040|     Argentina|2025-07-06|
|      2015|2025-07-06 08:55:...|   76|          |22cd5b46-62bc-487...|  EMAIL|   1011|     Argentina|2025-07-06|
|      2003|2025-07-06 08:54:...|   40|      NONE|300c1f8c-6687-441...|    SMS|   1002|       Germany|2025-07-06|
|      2003|2025-07-06 08:54:...|   48|          |f4a463db-7fdf-44a...|  EMAIL|   1033|       Germany|2025-07-06|
|      2003|2025-07-06 08:54:...|   18|          |2ff7ec01-6ea5-427...|    SMS|   1003| 

20

In [10]:
df_valid = spark.read.format("parquet").load("content/lake/silver/messages/data")
df_valid.show()
df_valid.count()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|      country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|      2009|2025-07-06 08:54:...|   21|  RECEIVED|59c1cf52-c13b-4b6...|  EMAIL|   1037|    Australia|2025-07-06|
|      2009|2025-07-06 08:54:...|   26|   CLICKED|9be7542b-27cd-402...|   CHAT|   1033|    Australia|2025-07-06|
|      2009|2025-07-06 08:54:...|   36|   CLICKED|e4e239cd-029f-4c9...|   CHAT|   1029|    Australia|2025-07-06|
|      2009|2025-07-06 08:55:...|   65|      SENT|f4a463db-7fdf-44a...|    SMS|   1041|    Australia|2025-07-06|
|      2010|2025-07-06 08:54:...|   38|      OPEN|7de85e38-0b0b-490...|  EMAIL|   1003|        Japan|2025-07-06|
|      2010|2025-07-06 08:55:...|   50|      SENT|820cbc69-7b7e-47b...|  EMAIL|   1045|        J

66

In [ ]:
corrupted_query.stop()
valid_query.stop()

## Checking data

In [15]:
# TODO (count messages da silver (corrupted e not corrupted) = mensagens bronze)


df_valid.count() + df_corrupted.count() == df.count()

False

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

# Q2:
- Which storage would you use and why? (database?, data lake?, kafka?)

